## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
!pip install --upgrade gmaps

# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

from google.colab import output
output.enable_custom_widget_manager()

In [19]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Olga,RU,2022-02-03 01:16:39,43.7444,135.2892,13.87,49,3,14.23,clear sky
1,1,Upington,ZA,2022-02-03 01:16:39,-28.4478,21.2561,71.98,94,100,5.75,overcast clouds
2,2,Poum,NC,2022-02-03 01:16:40,-20.2333,164.0167,80.15,73,0,0.45,clear sky
3,3,Cabra,ES,2022-02-03 01:16:40,37.4725,-4.4421,52.18,46,100,4.18,overcast clouds
4,4,Upernavik,GL,2022-02-03 01:13:09,72.7868,-56.1549,-8.97,67,100,8.77,overcast clouds


In [20]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
2,2,Poum,NC,2022-02-03 01:16:40,-20.2333,164.0167,80.15,73,0,0.45,clear sky
6,6,Areia Branca,BR,2022-02-03 01:16:41,-4.9561,-37.1369,81.03,76,41,15.01,scattered clouds
13,13,Avarua,CK,2022-02-03 01:06:44,-21.2078,-159.7750,84.25,79,75,18.41,broken clouds
15,15,Vao,NC,2022-02-03 01:16:44,-22.6667,167.4833,78.37,71,100,9.89,overcast clouds
16,16,Carnarvon,AU,2022-02-03 01:08:26,-24.8667,113.6333,87.93,48,6,5.06,clear sky
19,19,Sola,VU,2022-02-03 01:16:45,-13.8833,167.5500,81.57,75,100,9.28,overcast clouds
22,22,Labuhan,ID,2022-02-03 01:10:30,-6.8844,112.2051,80.65,79,81,1.03,broken clouds
23,23,Dingle,PH,2022-02-03 01:16:46,10.9995,122.6711,80.74,78,70,13.09,broken clouds
24,24,Itarema,BR,2022-02-03 01:14:26,-2.9248,-39.9167,79.18,79,100,10.07,overcast clouds
26,26,Arraial Do Cabo,BR,2022-02-03 01:11:58,-22.9661,-42.0278,75.25,88,100,4.25,overcast clouds


In [22]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID            0
City               0
Country            0
Date               0
Lat                0
Lng                0
Max Temp           0
Humidity           0
Cloudiness         0
Wind Speed         0
Current Weather    0
dtype: int64

In [23]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
2,2,Poum,NC,2022-02-03 01:16:40,-20.2333,164.0167,80.15,73,0,0.45,clear sky
6,6,Areia Branca,BR,2022-02-03 01:16:41,-4.9561,-37.1369,81.03,76,41,15.01,scattered clouds
13,13,Avarua,CK,2022-02-03 01:06:44,-21.2078,-159.7750,84.25,79,75,18.41,broken clouds
15,15,Vao,NC,2022-02-03 01:16:44,-22.6667,167.4833,78.37,71,100,9.89,overcast clouds
16,16,Carnarvon,AU,2022-02-03 01:08:26,-24.8667,113.6333,87.93,48,6,5.06,clear sky
...,...,...,...,...,...,...,...,...,...,...,...
688,688,Ballina,AU,2022-02-03 01:35:47,-28.8667,153.5667,75.16,93,75,6.91,light rain
689,689,Las Lomas,PA,2022-02-03 01:35:47,8.4333,-82.4000,78.21,83,20,2.30,few clouds
690,690,Nisia Floresta,BR,2022-02-03 01:33:16,-6.0911,-35.2086,82.78,84,89,7.34,overcast clouds
695,695,Bonoua,CI,2022-02-03 01:35:50,5.2741,-3.5950,78.42,91,6,2.64,clear sky


In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Poum,NC,80.15,clear sky,-20.2333,164.0167,
6,Areia Branca,BR,81.03,scattered clouds,-4.9561,-37.1369,
13,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,
15,Vao,NC,78.37,overcast clouds,-22.6667,167.4833,
16,Carnarvon,AU,87.93,clear sky,-24.8667,113.6333,
19,Sola,VU,81.57,overcast clouds,-13.8833,167.5500,
22,Labuhan,ID,80.65,broken clouds,-6.8844,112.2051,
23,Dingle,PH,80.74,broken clouds,10.9995,122.6711,
24,Itarema,BR,79.18,overcast clouds,-2.9248,-39.9167,
26,Arraial Do Cabo,BR,75.25,overcast clouds,-22.9661,-42.0278,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Poum,NC,80.15,clear sky,-20.2333,164.0167,Gîte kajeon
6,Areia Branca,BR,81.03,scattered clouds,-4.9561,-37.1369,Areia Branca Apart Hotel
13,Avarua,CK,84.25,broken clouds,-21.2078,-159.7750,Paradise Inn
15,Vao,NC,78.37,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
16,Carnarvon,AU,87.93,clear sky,-24.8667,113.6333,Hospitality Carnarvon


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))